In [252]:
import numpy as np
import heapq

# Q1

This 'jobs_greedy_algorithm.txt' file describes a set of jobs with positive and integral weights and lengths. It has the format

[number_of_jobs]

[job_1_weight] [job_1_length]

[job_2_weight] [job_2_length]

...

For example, the third line of the file is "74 59", indicating that the second job has weight 74 and length 59.

You should NOT assume that edge weights or lengths are distinct.

Your task in this problem is to run the greedy algorithm that schedules jobs in decreasing order of the difference (weight - length). Recall from lecture that this algorithm is not always optimal. IMPORTANT: if two jobs have equal difference (weight - length), you should schedule the job with higher weight first. Beware: if you break ties in a different way, you are likely to get the wrong answer. You should report the sum of weighted completion times of the resulting schedule --- a positive integer --- in the box below.

ADVICE: If you get the wrong answer, try out some small test cases to debug your algorithm (and post your test cases to the discussion forum).

In [293]:
with open('jobs_greedy_algorithm.txt','r') as f:
    lines = f.readlines()

jobs = list(map(lambda x: list(map(int, x.split())), lines))

In [324]:
weighted_jobs = list(map(lambda x: [x[1]-x[0], x[0], x[1]], jobs[1:]))

heapq.heapify(weighted_jobs)
tie_bucket = []
weighted_ct = 0
ct = 0

tmp_job = heapq.heappop(weighted_jobs)
tmp_weight = tmp_job[0]
tie_bucket.append(tmp_job)
count = 0
count2 = 1


while weighted_jobs != []:
    if weighted_jobs != []:
        tmp_job = heapq.heappop(weighted_jobs)
        while tmp_job[0] == tmp_weight:
            tie_bucket.append(tmp_job)
            count2 += 1
            if weighted_jobs != []:
                tmp_job = heapq.heappop(weighted_jobs)
            else:
                break
        tmp_weight = tmp_job[0]
    else:
        tmp_job = None
    
    if tie_bucket != []:
        tie_bucket2 = list(map(lambda x: [-x[1], x[2]], tie_bucket))
        tie_bucket = []
        heapq.heapify(tie_bucket2)

        while tie_bucket2 != []:
            tmp_job2 = heapq.heappop(tie_bucket2)
            count += 1
            #print(tmp_job2)
            ct += tmp_job2[1]
            weighted_ct += -tmp_job2[0]*ct
    
    if tmp_job is not None: 
        tie_bucket.append(tmp_job)
        count2 += 1

weighted_ct, count, count2

(69119377652, 10000, 10001)

# Q2

For this problem, use the same data set as in the previous problem.

Your task now is to run the greedy algorithm that schedules jobs (optimally) in decreasing order of the ratio (weight/length). In this algorithm, it does not matter how you break ties. You should report the sum of weighted completion times of the resulting schedule --- a positive integer --- in the box below.

In [286]:
weighted_jobs = list(map(lambda x: [-x[0]/x[1], x[0], x[1]], jobs[1:]))

heapq.heapify(weighted_jobs)
ct = 0
weighted_ct = 0
count = 0

weighted_jobs2 = []

while weighted_jobs != []:
    
    tmp_job = heapq.heappop(weighted_jobs)
    count += 1
    ct += tmp_job[2]
    weighted_ct += tmp_job[1]*ct

weighted_ct, count, ct

(67311454237, 10000, 510289)

# Q3 Prim's MST Algorithm

This 'MST.txt' file describes an undirected graph with integer edge costs. It has the format

[number_of_nodes] [number_of_edges]

[one_node_of_edge_1] [other_node_of_edge_1] [edge_1_cost]

[one_node_of_edge_2] [other_node_of_edge_2] [edge_2_cost]

...

For example, the third line of the file is "2 3 -8874", indicating that there is an edge connecting vertex #2 and vertex #3 that has cost -8874.

You should NOT assume that edge costs are positive, nor should you assume that they are distinct.

Your task is to run Prim's minimum spanning tree algorithm on this graph. You should report the overall cost of a minimum spanning tree --- an integer, which may or may not be negative --- in the box below.

IMPLEMENTATION NOTES: This graph is small enough that the straightforward O(mn) time implementation of Prim's algorithm should work fine. OPTIONAL: For those of you seeking an additional challenge, try implementing a heap-based version. The simpler approach, which should already give you a healthy speed-up, is to maintain relevant edges in a heap (with keys = edge costs). The superior approach stores the unprocessed vertices in the heap, as described in lecture. Note this requires a heap that supports deletions, and you'll probably need to maintain some kind of mapping between vertices and their positions in the heap.

In [206]:
with open('MST.txt', 'r') as f:
    lines = f.readlines()

lines = list(map(lambda x: list(map(int, x.split())), lines))

In [207]:
n, m =lines[0]
edges = list(map(lambda x: [x[2], x[0], x[1]], lines[1:]))

In [214]:
# naive implementaion

# initialize
V = set(range(1,n+1))
X = set()
T = []

e_cheapest = min(edges)
X.add(e_cheapest[1])
X.add(e_cheapest[2])
T.append(e_cheapest)

while X != V:
    c = np.infty
    for e in edges:
        if (e[1] in X) + (e[2] in X) == 1:
            if e[0] <= c:
                e_cheapest = e
                c = e[0]
        
    X.add(e_cheapest[1])
    X.add(e_cheapest[2])
    T.append(e_cheapest)

In [215]:
len(T)

499

In [218]:
total_cost = sum(list(map(lambda x: x[0], T)))
total_cost

-3612829

In [249]:
# heap_inplementation

# initialize
G = list(map(lambda x: [], range(n+1)))
any(map(lambda x: G[x[2]].append([x[0], x[1]]), edges))
any(map(lambda x: G[x[1]].append([x[0], x[2]]), edges))

V = set(range(1,n+1))
X = set()
T = []

e_cheapest = min(edges)
X.add(e_cheapest[1])
X.add(e_cheapest[2])
T.append(e_cheapest)
heap_v = list(map(lambda x: [np.inf, x, x], V-X))

def update_cheapest_edge(X, G,heap_v):
    for i in range(len(heap_v)):
        v1 = heap_v[i][1]
        v2 = heap_v[i][2]
        
        if (v1 not in X) + (v2 not in X) == 1:
            if v1 not in X:
                v3 = v1
            else: 
                v3 = v2
        elif v1 == v2:
            v3 = v1
        
        c = np.inf
        v4 = 0 
        for e in G[v3]:
            if e[1] in X and e[0] < c:
                v4 = e[1]
                c  = e[0]
        
        if c != np.inf: 
            heap_v[i] = [c, v3, v4]
            #print(heap_v[i])
    
    heapq.heapify(heap_v)
    return        

update_cheapest_edge(X, G, heap_v)

#print(heap_v)

while X != V:
    e = heapq.heappop(heap_v)
    #print(e)
    if e[1] in X:
        X.add(e[2])
    elif e[2] in X:
        X.add(e[1])
    else:
        print('booo')
    
    update_cheapest_edge(X, G, heap_v)
    T.append(e)

In [250]:
len(T)

499

In [251]:
total_cost = sum(list(map(lambda x: x[0], T)))
total_cost

-3612829

The following problems are for those of you looking to challenge yourself beyond the required problem sets and programming questions. They are completely optional and will not be graded. While they vary in level, many are pretty challenging, and we strongly encourage you to discuss ideas and approaches with your fellow students on the "Theory Problems" discussion forum.

1. Consider a connected undirected graph GG with not necessarily distinct edge costs. Consider two different minimum-cost spanning trees of G, T and $T'$. Is there necessarily a sequence of minimum-cost spanning trees $T = T_0, T_1, T_2, \ldots, T_r = T'$ with the property that each consecutive pair $T_i,T_{i+1}$ of MSTs differ by only a single edge swap? Prove the statement or exhibit a counterexample.

2. Consider the following algorithm. The input is a connected undirected graph with edge costs (distinct, if you prefer). The algorithm proceeds in iterations. If the current graph is a spanning tree, then the algorithm halts. Otherwise, it picks an arbitrary cycle of the current graph and deletes the most expensive edge on the cycle. Is this algorithm guaranteed to compute a minimum-cost spanning tree? Prove it or exhibit a counterexample.

3. Consider the following algorithm. The input is a connected undirected graph with edge costs (distinct, if you prefer). The algorithm proceeds in phases. Each phase adds some edges to a tree-so-far and reduces the number of vertices in the graph (when there is only 1 vertex left, the MST is just the empty set). In a phase, we identify the cheapest edge $e_v$ incident on each vertex $v$ of the current graph. Let $F = \{ e_v \}$ be the collection of all such edges in the current phase. Obtain a new (smaller) graph by contracting all of the edges in FF --- so that each connected component of $F$ becomes a single vertex in the new graph --- discarding any self-loops that result. Let $T$ denote the union of all edges that ever get contracted in a phase of this algorithm. Is $T$ guaranteed to be a minimum-cost spanning tree? Prove it or exhibit a counterexample.